#CSE 101: Computer Science Principles
####Stony Brook University
####Kevin McDonnell (ktm@cs.stonybrook.edu)
##Module 20: Cryptography


### Note

* See the course website for the [lecture slides](https://drive.google.com/file/d/1-5WhxD_h6fRQJd0_kNyUM_wM__L7cSSW/view?usp=sharing), which are separate from this Colab file




### Caesar Cipher

In [ ]:
def caesar_encrypt(plaintext, k):
    ciphertext = ''
    for ch in plaintext:
        if ch.isupper():
            replacement = (ord(ch) - ord('A') + k) % 26 + ord('A')
            ciphertext += chr(replacement)
        elif ch.islower():
            replacement = (ord(ch) - ord('a') + k) % 26 + ord('a')
            ciphertext += chr(replacement)
        else:
            ciphertext += ch
    return ciphertext

caesar_encrypt("What's a Seawolf?", 5)  # use double-quotes to deal with the single quote

"Bmfy'x f Xjfbtqk?"

In [ ]:
def caesar_decrypt(ciphertext, k):
    plaintext = ''
    for ch in ciphertext:
        if ch.isupper():
            replacement = (ord(ch) - ord('A') - k + 26) % 26 + ord('A')
            plaintext += chr(replacement)
        elif ch.islower():
            replacement = (ord(ch) - ord('a') - k + 26) % 26 + ord('a')
            plaintext += chr(replacement)
        else:
            plaintext += ch
    return plaintext

caesar_decrypt("Bmfy'x f Xjfbtqk?", 5)

"What's a Seawolf?"

### Multiplicative Cipher

In [ ]:
def multiplicative_encrypt(plaintext, k):
    ciphertext = ''
    for ch in plaintext:
        if ch.isupper():
            replacement = ((ord(ch) - ord('A')) * k) % 26 + ord('A')
            ciphertext += chr(replacement)
        elif ch.islower():
            replacement = ((ord(ch) - ord('a')) * k) % 26 + ord('a')
            ciphertext += chr(replacement)
        else:
            ciphertext += ch
    return ciphertext

multiplicative_encrypt("What's a Seawolf?", 5)

"Gjar'm a Muagsdz?"

In [ ]:
import string

reverse_mapping = {}
decrypt_key = -1
def multiplicative_decrypt(ciphertext, k):
    global reverse_mapping, decrypt_key
    if k != decrypt_key:
        decrypt_key = k
        encrypted_letters = [multiplicative_encrypt(letter, k) for letter in 
                           string.ascii_letters]
        reverse_mapping = {encrypted_letter: letter for letter, encrypted_letter in                
                         zip(string.ascii_letters, encrypted_letters)}
    plaintext = ''
    for ch in ciphertext:
        if ch in reverse_mapping:
            plaintext += reverse_mapping[ch]
        else:
            plaintext += ch
    return plaintext

multiplicative_decrypt("Gjar'm a Muagsdz?", 5)

"What's a Seawolf?"

### Affine Cipher

In [ ]:
def affine_encrypt(plaintext, a, b):
    ciphertext = ''
    for ch in plaintext:
        if ch.isupper():
            replacement = ((ord(ch) - ord('A')) * a + b) % 26 + ord('A')
            ciphertext += chr(replacement)
        elif ch.islower():
            replacement = ((ord(ch) - ord('a')) * a + b) % 26 + ord('a')
            ciphertext += chr(replacement)
        else:
            ciphertext += ch
    return ciphertext

affine_encrypt("What's a Seawolf?", 5, 2)

"Ilct'o c Owciufb?"

The "hacky" decryption algorithm for the affine cipher is essentially the same as for the multiplicative cipher, and so is not included here.

### Railfence Cipher

In [ ]:
def next_row(row, step, num_rows):
    if row == 0:
        step = 1
    elif row == num_rows - 1:
        step = -1
    row += step
    return row, step

def railfence_encrypt(plaintext, num_rows):
    row = 0
    step = 1
    # create num_rows empty strings in a list
    rows = [''] * num_rows
    for ch in plaintext:
        rows[row] += ch
        row, step = next_row(row, step, num_rows)
    return ''.join(rows)

print(railfence_encrypt('STONYBROOKUNIV', 3))
print(railfence_encrypt('STONYBROOKUNIV', 4))

SYOITNBOKNVORU
SRITBONVOYOUNK


In [ ]:
def railfence_decrypt(ciphertext, num_rows):
    grid = []
    for i in range(num_rows):
        grid += [[''] * len(ciphertext)] # a grid of empty strings

   # set up the grid, placing a None value 
   # where each letter will go
    row = 0
    step = 1
    for col in range(len(ciphertext)):
        grid[row][col] = None
        row, step = next_row(row, step, num_rows)
    next_char_index = 0
    for row in range(num_rows):
        for col in range(len(ciphertext)):
            if grid[row][col] is None:
                grid[row][col] = ciphertext[next_char_index]
                next_char_index += 1
    
    plaintext = ''
    row = 0
    step = 1
    for col in range(len(ciphertext)):
        plaintext += grid[row][col]
        row, step = next_row(row, step, num_rows)
    return plaintext

print(railfence_decrypt('SYOITNBOKNVORU', 3))
print(railfence_decrypt('SRITBONVOYOUNK', 4))

STONYBROOKUNIV
STONYBROOKUNIV


### The Vigenère Cipher

In [ ]:
def vigenere_encrypt(plaintext, keyword):
    # duplicate the keyword as many times as needed
    keyword = keyword * (len(plaintext) // len(keyword) + 1)
    # convert plaintext letters to numbers
    plaintext_nums = [ord(ch) - ord('A') for ch in plaintext]
    # convert keyword letters to numbers
    keyword_nums = [ord(ch) - ord('A') for ch in keyword]
    # generate ciphertext
    ciphertext = ''
    for i in range(len(plaintext)):
        # add the two numerical codes and map the sum (mod 26) 
        # back to a letter
        ciphertext += chr((plaintext_nums[i]+keyword_nums[i]) % 26 + ord('A'))
    return ciphertext

print(vigenere_encrypt('FUNCTION', 'JOKE'))

OIXGCWYR


In [ ]:
def vigenere_decrypt(ciphertext, keyword):
    # duplicate the keyword as many times as needed
    keyword = keyword * (len(ciphertext) // len(keyword) + 1)
    # convert ciphertext letters to numbers
    ciphertext_nums = [ord(ch) - ord('A') for ch in ciphertext]
    # convert keyword letters to numbers
    keyword_nums = [ord(ch) - ord('A') for ch in keyword]
    # generate plaintext   
    plaintext = ''
    for i in range(len(ciphertext)):
        # subtract keyword num from ciphertext num, add 26
        # and map difference (mod 26) back to a letter
        plaintext += chr((ciphertext_nums[i]-keyword_nums[i] + 26) % 26 + ord('A'))
    return plaintext

print(vigenere_decrypt('OIXGCWYR', 'JOKE'))

FUNCTION
